# Sedimentation in a Fluid

The purpose of this tutorial is to demonstrate how hydrodynamic interactions can have a dramatic impact on the overall dynamics of a molecular dynamics (MD) system.

We will set up a simple semi-two-dimensional system of sedimenting particles and simulate it first with Langevin dynamics.
Susequently, the system will be coupled to a lattice-Boltzmann fluid instead.

Both scenarios will then be compared by visualizing the data.

In [ ]:
import espressomd
import espressomd.lb
import espressomd.lbboundaries
import espressomd.shapes

# imports for data handling, plotting, and progress bar
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

We intend to use a purely repulsive WCA interaction potential for the particles, with the following parameters:

In [ ]:
lj_sigma = 1
lj_epsilon = 1
lj_cutoff = 2**(1. / 6) * lj_sigma

The intiial particles positions will be chosen to form a two-dimensional hexagonal Bravais lattice structure.
The spacing we use is large enough for the particles not to interact initially. If particles are positioned closer particles would rearrange to minimize the interaction energy, breaking up the initial structure.
We setup the simulation box such that the particle structure is periodically repeated in $x$-direction.

In [ ]:
def hexagonal_lattice(n_rows=5, parts_per_row=5, spacing=1):
    positions = np.array([[x + 0.12 + (0.5 if y % 2 == 1 else 0),
                           y * np.sqrt(3) / 2, 0]
                          for x in range(parts_per_row)
                          for y in range(n_rows)]) * spacing
    return positions

Now, we are ready to define the system parameters and initialize the simulation system.

**possible task: set up WCA potential**

In [ ]:
# parameters

# lattice spacing and number of lattice rows to use
spacing = lj_cutoff
n_rows = 10

# system size in units of lattice spacing
n_height = 40
n_width = 20
n_depth = 2

# resulting box geometry
box_height = n_height * spacing
box_width = n_width * spacing
box_depth = n_depth * spacing

# system setup
system = espressomd.System(box_l=[box_width, box_height, box_depth])
system.time_step = 0.01
system.cell_system.skin = 0.4

# add non-bonded WCA interaction
system.non_bonded_inter[0, 0].lennard_jones.set_params(
    epsilon=lj_epsilon, sigma=lj_sigma, cutoff=lj_cutoff, shift="auto")

We add a wall constraint on bottom and top of the simulation box, respectively.

**possible task: set up WCA wall constaint (wall shapes provided)**

In [ ]:
# create wall shapes bottom (b) and top (t)
wall_shape_b = espressomd.shapes.Wall(normal=[0, 1, 0], dist=1)
wall_shape_t = espressomd.shapes.Wall(
    normal=[0, -1, 0], dist=-(box_height - 1))

# add wall constraints
for wall_shape in [wall_shape_b, wall_shape_t]:
    system.constraints.add(shape=wall_shape, particle_type=0)

In [ ]:
# prepare for sampling
sampling_steps = 700

initial_positions = hexagonal_lattice(n_rows, n_width, spacing)
# total number of particles
n_parts = n_rows * n_width
assert initial_positions.shape == (n_parts, 3)

# we introduce a small imperfection into the initial lattice structure
initial_positions[-1, 0] -= 0.3 * spacing

# shift initial positions to the top
y_max = np.amax(initial_positions[:, 1])
initial_positions += (box_height - 1 - lj_cutoff) - y_max

# data structures to hold particle trajectories
data_lv = np.full((sampling_steps, n_parts, 3), np.nan)
data_lb = np.full_like(data_lv, np.nan)

We will set an external force acting on all particles. You can take this to model the effect of gravity, for example.

In [ ]:
f_gravity = [0, -3, 0]

Now, we start with sampling using the Langevin thermostat (with temperature $T=0$).

**possible tasks: set Langevin thermostat, add particles**

In [ ]:
# Langevin simulation
system.thermostat.set_langevin(kT=0., gamma=15, seed=12)

parts = system.part.add(pos=initial_positions, ext_force=[f_gravity] * n_parts)

system.integrator.run(0)

for step in tqdm(range(sampling_steps)):
    data_lv[step, :] = parts.pos_folded    
    system.integrator.run(25)
data_lv[-1, :] = parts.pos_folded


Now we want to sample the same system, but with a coupled lattice-Boltzmann fluid. We first reset the particles to their initial positions and remove any particle velocities.
Then we set up the LB fluid. The wall constraints that were previously added have to be also registered as LB boundaries.

**possible tasks: set up LB fluid, add lbboundaries**

In [ ]:
# LB simulation cleanup and setup
parts.pos = initial_positions
parts.v = [0, 0, 0]
system.thermostat.turn_off()

lbf = espressomd.lb.LBFluid(agrid=spacing, dens=1.0, visc=1.0, tau=system.time_step, kT=0)
system.actors.add(lbf)
system.thermostat.set_lb(LB_fluid=lbf, seed=123, gamma=15)

# add LB boundaries at walls
for wall_shape in [wall_shape_b, wall_shape_t]:
    no_slip_wall = espressomd.lbboundaries.LBBoundary(
        shape=wall_shape, velocity=[0, 0, 0])
    system.lbboundaries.add(no_slip_wall)

In [ ]:
# numpy array to hold flowfield data
data_flowfield = np.full((sampling_steps, n_height, n_width, 3), np.nan)

# return n_x x n_x array of lattice velocities, averaged in z-direction 
def mean_flowfield(xs=n_width, ys=n_height, zs=n_depth):
    dataframe = np.array([[[lbf[x, y, z].velocity for z in range(zs)] for x in range(xs)] 
                          for y in range(ys)])
    return np.mean(dataframe, axis=2)

for step in tqdm(range(sampling_steps)):
    data_lb[step, :] = parts.pos_folded
    data_flowfield[step, :] = mean_flowfield()    
    system.integrator.run(25)
data_lb[-1, :] = parts.pos_folded
data_flowfield[-1, :] = mean_flowfield()

Now let's visualize the data. First some imports and definitions for inline visualization.

In [ ]:
import matplotlib.animation as animation
import tempfile
import base64

from matplotlib.quiver import Quiver

VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""


def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with tempfile.NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264'])
            with open(f.name, "rb") as g:
                video = g.read()
        anim._encoded_video = base64.b64encode(video).decode('ascii')
        plt.close(anim._fig)
    return VIDEO_TAG.format(anim._encoded_video)


animation.Animation._repr_html_ = anim_to_html

# set ignore 'divide' and 'invalid' errors
# these occur when plotting the flowfield containing a zero velocity
np.seterr(divide='ignore', invalid='ignore')

And now the actual visualization code.

In [ ]:
# setup figure and prepare axes
fig = plt.figure(figsize=(2 * 5, 5 / box_width * box_height))
gs = fig.add_gridspec(1, 2, wspace=0.1)
(ax1, ax2) = gs.subplots(sharey=True)

ax1.set_title("Langevin")
ax1.set_xlim((0, box_width))
ax1.set_ylim((0, box_height))

ax2.set_title("LB Fluid")
ax2.set_xlim((0, box_width))
ax2.set_ylim((0, box_height))

# draw walls
for ax in [ax1, ax2]:
    ax.hlines((1, box_height-1), 0, box_width, color="gray")

# create meshgrid for quiver plot
xs = np.array([x for x in range(n_width)]) * spacing
ys = np.array([y for y in range(n_height)]) * spacing
X, Y = np.meshgrid(xs, ys)

# initialize plot objects
lb_ff = ax2.quiver(X, Y, data_flowfield[0, :, :, 0],
                   data_flowfield[0, :, :, 1])
lb_particles, = ax2.plot([], [], 'o')
lv_particles, = ax1.plot([], [], 'o')

def draw_frame(t):
    # manually remove Quivers from ax2
    for artist in ax2.get_children():
        if isinstance(artist, Quiver):
            artist.remove()
    
    # draw new quivers
    lb_ff = ax2.quiver(X, Y, data_flowfield[t, :, :, 0],
                       data_flowfield[t, :, :, 1])
    
    # draw particles
    lv_particles.set_data(data_lv[t, :, 0], data_lv[t, :, 1])
    lb_particles.set_data(data_lb[t, :, 0], data_lb[t, :, 1])

    return [lv_particles, lb_particles, lb_ff]

animation.FuncAnimation(fig, draw_frame, frames=sampling_steps, blit=True, interval=0, repeat=False)